<a target="_blank" href="https://nbviewer.org/github/a5dur/qsv-whirlwind-tour/blob/main/Whirlwindtour.ipynb">
    <img src="https://img.shields.io/badge/nbviewer-nbviewer.org-orange?logo=jupyter&label=View%20online" alt="View online on nbviewer.org">
</a>
<a target="_blank" href="https://github.com/jqnatividad/qsv/blob/master/contrib/notebooks/intro-to-count.ipynb">
    <img src="https://img.shields.io/badge/github-jqnatividad%2Fqsv-white?logo=github&label=Source%20Code" alt="View the source code on GitHub">
</a>


### A whirlwind tour

> ℹ️ **NOTE:** This tour is primarily targeted to Linux and macOS users. Though qsv works on Windows, the tour
assumes basic knowledge of command-line piping and redirection, and uses other command-line tools (curl, tee, head, etc.)
that are not installed by default on Windows.

Let's say you're playing with some data from the
[Data Science Toolkit](https://github.com/petewarden/dstkdata), which contains
several CSV files. Maybe you're interested in the population counts of each
city in the world. So grab the 124MB, 2.7M row CSV file and start examining it:


In [1]:
!qsv headers wcp.csv

1   Country
2   City
3   AccentCity
4   Region
5   Population
6   Latitude
7   Longitude


The next thing you might want to do is get an overview of the kind of data that
appears in each column. The `stats` command will do this for you:

!qsv stats wcp.csv | qsv table

Wow! That was fast! It took just 1.3 seconds to compile all that.[^1] One reason for qsv's speed
is that ***it mainly works in "streaming" mode*** - computing statistics as it "streams"
the CSV file line by line. This also means it can gather statistics on arbitrarily large files,
as it does not have to load the entire file into memory.[^2]

But can we get more summary statistics? What's the variance, the modes, the distribution (quartiles), 
and the cardinality of the data?  No problem. That's why `qsv stats` has an `--everything` option to 
compute these more "expensive" stats. Expensive - as these extended statistics can only be computed at 
the cost of loading the entire file into memory.


In [8]:
!qsv stats wcp.csv --everything | qsv table

field       type     sum            min           max         range     min_length  max_length  mean        stddev       variance          nullcount  sparsity  mad      lower_outer_fence  lower_inner_fence  q1       q2_median  q3       iqr      upper_inner_fence  upper_outer_fence  skewness  cardinality  mode         mode_count  mode_occurrences  antimode                                                                                                 antimode_count  antimode_occurrences
Country     String                  ad            zw                    2           2                                                      0          0                                                                                                                                              231          ru           1           176934            cc,nf,pn,tf,tk                                                                                           5               1
City        String                   

> ℹ️ **NOTE:** The `qsv table` command takes any CSV data and formats it into aligned columns
using [elastic tabstops](https://github.com/BurntSushi/tabwriter). You'll
notice that it even gets alignment right with respect to Unicode characters.


So, this command took 3.22 seconds to run on my machine, but we can speed
it up by creating an index and re-running the command:

```
qsv index wcp.csv
qsv stats wcp.csv --everything | qsv table
```

Which cuts it down to 1.95 seconds - 1.65x faster! (And creating the 21.6mb index took 0.27 seconds. 
What about the first `stats` without `--everything`? From 1.3 seconds to 0.16 seconds with an index - 8.25x faster!)

Notably, the same type of "statistics" command in another
[CSV command line toolkit](https://csvkit.readthedocs.io/)
takes about 10 seconds to produce a *subset* of statistics on the same data set. [Visidata](https://visidata.org)
takes much longer - ~1.5 minutes to calculate a *subset* of these statistics with its Describe sheet. 
Even python [pandas'](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) 
`describe(include="all"))` took 12 seconds to calculate a *subset* of qsv's "streaming" statistics.[^3]

This is another reason for qsv's speed. Creating an index accelerated statistics gathering as it enables 
***multithreading & fast I/O***.

**For multithreading** - running `stats` with an index was 8.25x faster because it divided the file into 
16 equal chunks[^1] with ~170k records each, then running stats on each chunk in parallel across 16 
logical processors and merging the results in the end. It was "only" 8x, and not 16x faster as there is 
some overhead involved in multithreading. 

**For fast I/O** - let's say you wanted to grab the last 10 records:

In [12]:
!qsv count --human-readable wcp.csv

2,699,354


In [14]:
!qsv slice wcp.csv --start -10 | qsv table

Country  City               AccentCity         Region  Population  Latitude     Longitude
zw       zibalonkwe         Zibalonkwe         06                  -19.8333333  27.4666667
zw       zibunkululu        Zibunkululu        06                  -19.6666667  27.6166667
zw       ziga               Ziga               06                  -19.2166667  27.4833333
zw       zikamanas village  Zikamanas Village  00                  -18.2166667  27.95
zw       zimbabwe           Zimbabwe           07                  -20.2666667  30.9166667
zw       zimre park         Zimre Park         04                  -17.8661111  31.2136111
zw       ziyakamanas        Ziyakamanas        00                  -18.2166667  27.95
zw       zizalisari         Zizalisari         04                  -17.7588889  31.0105556
zw       zuzumba            Zuzumba            06                  -20.0333333  27.9333333
zw       zvishavane         Zvishavane         07      79876       -20.3333333  30.0333333


`qsv count` took 0.006 seconds and `qsv slice`, 0.017 seconds! These commands are *instantaneous* 
with an index because for `count` - the index already precomputed the record count, and with `slice`,
*only the sliced portion* has to be parsed - because an index allowed us to jump directly to that 
part of the file. It didn't have to scan the entire file to get the last 10 records. For comparison,
without an index, it took 0.25 (41x slower) and 0.66 (39x slower) seconds respectively.

> ℹ️ **NOTE:** Creating/updating an index itself is extremely fast as well. If you want
qsv to automatically create and update indices, set the environment var `QSV_AUTOINDEX`.

Okay, okay! Let's switch gears and stop obsessing over how fast :rocket: qsv is... let's go back to exploring :mag_right:
the data set.

Hmmmm... the Population column has a lot of null values. How pervasive is that?
First, let's take a look at 10 "random" rows with `sample`. We use the `--seed` parameter
so we get a reproducible random sample. And then, let's display only the Country,
AccentCity and Population columns with the `select` command.

In [15]:
!qsv sample --seed 42 10 wcp.csv | qsv select Country,AccentCity,Population | qsv table

Country  AccentCity              Population
co       Guamal                  5026
af       Jalat Kalay             
ro       Caragheorghevici        
mm       Padauk                  
tr       Vakifbeloren            
mx       Chupadero de Alcaparra  
ir       Gangar Kola             
tg       Folo                    
pe       Huancas                 
pk       Dilawanwala             


Whoops! The sample we got don't have population counts. It's quite pervasive. Exactly how many cities have empty (NULL) population counts?

In [16]:
!qsv frequency wcp.csv --limit 3 | qsv table

field       value        count
Country     ru           176934
Country     us           141989
Country     cn           117508
City        san jose     313
City        san antonio  310
City        santa rosa   288
AccentCity  San Antonio  307
AccentCity  Santa Rosa   288
AccentCity  Santa Cruz   268
Region      04           143900
Region      02           127736
Region      03           105455
Population  (NULL)       2652350
Population  2310         12
Population  983          11
Latitude    50.8         1128
Latitude    50.95        1076
Latitude    50.6         1043
Longitude   23.1         590
Longitude   23.2         586
Longitude   23.05        575


(The `qsv frequency` command builds a frequency table for each column in the
CSV data. This one only took 1.8 seconds.)

So it seems that most cities do not have a population count associated with
them at all (2,652,350 to be exact). No matter — we can adjust our previous 
command so that it only shows rows with a population count:


In [19]:
!qsv search --select Population '[0-9]' wcp.csv | qsv sample --seed 42 10 | qsv select Country,AccentCity,Population | tee sample.csv | qsv table

'tee' is not recognized as an internal or external command,
operable program or batch file.


> ℹ️ **NOTE:** The `tee` command reads from standard input and writes 
to both standard output and one or more files at the same time. We do this so 
we can create the `sample.csv` file we need for the next step, and pipe the 
same data to the `qsv table` command.<br/>Why create `sample.csv`? Even though qsv is blazing-fast, we're just doing an 
initial investigation and a small 10-row sample is all we need to try out and
compose the different CLI commands needed to wrangle the data.


Erk. Which country is `sv`? What continent? No clue, but [datawookie](https://github.com/datawookie) 
has a CSV file called `country-continent.csv`.


```
$ curl -L https://raw.githubusercontent.com/datawookie/data-diaspora/master/spatial/country-continent-codes.csv > country_continent.csv
$ qsv headers country_continent.csv
1 # https://datahub.io/JohnSnowLabs/country-and-continent-codes-list
```

Huh!?! That's not what we we were expecting. But if you look at the country-continent.csv file, it starts with a comment with the # character.

In [1]:
!qsv count wcp.csv

2699354
